In [1]:
# 이제 필요없는 거는 날릴 때도 됐다.

In [20]:
#한글 폰트 사용
from matplotlib import font_manager,rc
#폰트 경로
font_path = "C:/Users/user/AppData/Local/Microsoft/Windows/Fonts/NanumBarunGothic.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()

In [21]:
import pandas as pd
import numpy as np
import keras
import pymysql
from sklearn import preprocessing

In [22]:
# db 연결

conn=pymysql.connect(host='database-1.cani4y1t2hfg.ap-northeast-2.rds.amazonaws.com',port=3306,
                    user='slows14tem',password='!wjdgy0117',db='bigdata', cursorclass=pymysql.cursors.DictCursor)
cursor = conn.cursor()
sql = "select * from original"
cursor.execute(sql)
result = cursor.fetchall()
conn.close()
train = pd.DataFrame(result)


In [23]:
train
train.columns

Index(['id', 'cheongguseobeonho', 'no', 'subject_', 'machinery', 'assembly',
       'items', 'part1', 'Part2', 'key1', 'key2', 'leadtime', 'cheongguryang',
       'gyeonjeok', 'gyeonjeoksuryang', 'gyeonjeokhwapye', 'gyeonjeokdanga',
       'baljubeonho', 'baljucheo', 'balju', 'baljusuryang', 'baljugeumaek',
       'dt', 'miipgogigan', 'changgoipgo', 'changgoipgosuryang', 'control_no',
       'ipgochanggo', 'changgochulgo', 'changgochulgosuryang', 'chulgoseonbak',
       'chulgounbanseon', 'seonbagipgo', 'seonbagipgosuryang', 'wallyo_yeobu'],
      dtype='object')

In [24]:
label_encoder = preprocessing.LabelEncoder()

In [25]:
'''
그러니까 이걸 왜 골랐는지 설명을 해라.
subject_ 이런 거 고를 수 있다 key2를 분석하는데 필요한 engine같은 단어가 있잖아.
machinery 마찬가지고.
assembly에도 그런 key2와 유사한 키워드가 포함되어 있다.
part1 뺄 수 있을 것같다. 그런 차원에서 본다면 큰 의미가 없지 않나.
예전에 R에서 넣어봤을 때 물론 넣은게 accuracy가 더 높긴 했다.
'''
train['subject'] = label_encoder.fit_transform(train['subject_'])
train['machinery']= label_encoder.fit_transform(train['machinery'])
train['assembly']= label_encoder.fit_transform(train['assembly'])
train['items']= label_encoder.fit_transform(train['items'])

In [26]:
columns= ['machinery'  , 'assembly' , 'items', 'subject', 'key2']
features= ['machinery'  , 'assembly' , 'items', 'subject' ]
target= ['key2']

data_Encode 에 해당 컬럼들을 넣는다.

In [27]:
data_Encode= train[columns]
data_Encode

,machinery,assembly,items,subject,key2
0,109,295,4906,953,COOLER
1,114,1201,3733,1673,COOLER
2,114,220,5995,1674,COOLER
3,109,83,700,10,COOLER
4,109,251,4736,524,COOLER
...,...,...,...,...,...
21079,64,488,2839,258,CYLINDER
21080,64,488,2838,256,CYLINDER
21081,0,919,4200,576,BELT
21082,5,1231,1601,237,CYLINDER


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
# y에 key2가 들어있는 거임.
# 이거를 키려면 numpy가 1.24.0 미만이어야 됨. pip install "numpy<1.24.0"
X_tr, X_val, y_tr, y_val = train_test_split(data_Encode[features], data_Encode[target], stratify = data_Encode[target], test_size= 0.3)

In [30]:
from pycaret.classification import *

In [31]:
# train, validation
TR = pd.concat([X_tr, y_tr], axis=1)
VAL = pd.concat([X_val, y_val], axis=1)

환경설정

In [32]:
clf= setup(data=TR, target='key2', use_gpu=True, categorical_features=['machinery'  , 'assembly' , 'items', 'subject'])
# categorical을 안 주면 label을 숫자로 주니까 쓸 수가 없음
# clf= setup(data=TR, target='key2_Encode', use_gpu=True)

,Description,Value
0,session_id,8970
1,Target,key2
2,Target Type,Multiclass
3,Label Encoded,"ADAPTER: 0, ANODES: 1, ARM: 2, BATTERY: 3, BEARING: 4, BELT: 5, BINO: 6, BLOCK: 7, BOLT: 8, BRAKE: 9, BUSHING: 10, CARD: 11, CARTRIDGE: 12, CLAMP: 13, CLIP: 14, COMPRESSOR: 15, CONNECTING ROD: 16, CONTROL: 17, COOLER: 18, COVER: 19, CYLINDER: 20, DAMPER: 21, DISC: 22, FILTER: 23, GASKET: 24, GAUGE: 25, GEAR: 26, GENERATOR: 27, HARNESS: 28, HOSE: 29, INJECTOR: 30, KEY: 31, LINE: 32, LINK: 33, LIQUID: 34, MANIFOLD: 35, MONITOR: 36, MOTOR: 37, NET: 38, NUT: 39, O-RING: 40, PIN: 41, PIPE: 42, PISTON: 43, PLATE: 44, PUMP: 45, SCREW: 46, SEAT: 47, SHAFT: 48, SPACER: 49, SPRING: 50, SWITCH: 51, TOOL: 52, TRANSFORMER: 53, TUBE: 54, TURBINE: 55, UNIT: 56, VALVE: 57, WASHER: 58, WINCH: 59, WIRE: 60"
4,Original Data,"(14758, 5)"
5,Missing Values,False
6,Numeric Features,0
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


BEST 5 출력

In [33]:
# top5 = compare_models(sort='Accuracy', n_select=10)
# 점심시간 전에 model blending
top5 = compare_models(sort='Accuracy', n_select=5, fold=5)
blended_models = blend_models(top5, fold=5)
finalized_models = finalize_model(blended_models)
save_model(finalized_models, './original_model3/blend')


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.6530,0.0000,0.5762,0.6610,0.6491,0.6254,0.6261,11.6860
ridge,Ridge Classifier,0.6517,0.0000,0.5950,0.6594,0.6472,0.6228,0.6238,2.9820
et,Extra Trees Classifier,0.5914,0.8744,0.5239,0.5948,0.5861,0.5588,0.5594,17.0780
gbc,Gradient Boosting Classifier,0.5841,0.8705,0.5284,0.6873,0.5896,0.5343,0.5583,3259.6140
rf,Random Forest Classifier,0.5833,0.8841,0.5023,0.5865,0.5760,0.5486,0.5496,12.2580
lda,Linear Discriminant Analysis,0.5823,0.8359,0.5211,0.6510,0.5950,0.5458,0.5494,119.5360
lr,Logistic Regression,0.5767,0.9088,0.4284,0.6130,0.5636,0.5329,0.5395,25.1180
dt,Decision Tree Classifier,0.5696,0.7680,0.4979,0.5779,0.5656,0.5349,0.5356,14.9920
nb,Naive Bayes,0.4646,0.7290,0.5056,0.5238,0.4786,0.4368,0.4385,13.2220
knn,K Neighbors Classifier,0.4466,0.8084,0.3748,0.4888,0.4483,0.4033,0.4048,72.0260


IntProgress(value=0, description='Processing: ', max=6)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:30:52
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Fold,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


KeyboardInterrupt: 

In [ ]:
predictions_blend = predict_model(data=VAL, estimator=finalized_models)
predictions_blend

In [ ]:
'''
https://bigdaheta.tistory.com/53
교수님 말씀은, 니가 평가지표에 대해 제대로 알고 있어야 지도할 게 있다는 건데,
사실 key2, 지금 이 상태에서 분류를 맞추는 데 있어서는 accuracy가 높은 게 장땡아닌가 싶다.
정밀도는, false를 true로 잘못 예측하는 것이 critical할 때 검토할 필요가 있는 지표라고 하는데, 이 경우에도 ridge가 가장 높은 퍼포먼스를 보여줬었다 part1 있을 때는.
지금은 part1 빼버리고 subject를 넣어서 또 모델이 바뀔 것같은데 어떻게 될지는 지켜봐야지..
'''

In [ ]:
lr1 = create_model('lr')
finalize_model(lr1)
save_model(lr1, './original_model3/lr1')
prediction_lr = predict_model(lr1, VAL)
prediction_lr

In [ ]:
knn1 = create_model('knn')
finalize_model(knn1)
save_model(knn1, './original_model3/knn1')
prediction_knn = predict_model(knn1, VAL)
prediction_knn

In [ ]:
nb1 = create_model('nb')
finalize_model(nb1)
save_model(nb1, './original_model3/knn1')
prediction_nb = predict_model(nb1, VAL)
prediction_nb

In [ ]:
dt1 = create_model('dt')
finalize_model(dt1)
save_model(dt1, './original_model3/knn1')
prediction_dt = predict_model(dt1, VAL)
prediction_dt

In [ ]:
svm1 = create_model('svm')
finalize_model(svm1)
save_model(svm1, './original_model3/svm1')
prediction_svm = predict_model(svm1, VAL)
prediction_svm

In [ ]:
rbfsvm1 = create_model('rbfsvm')
finalize_model(rbfsvm1)
save_model(rbfsvm1, './original_model3/rbfsvm1')
prediction_rbfsvm = predict_model(rbfsvm1, VAL)
prediction_rbfsvm

In [ ]:
gpc1 = create_model('gpc')
finalize_model(gpc1)
save_model(gpc1, './original_model3/gpc1')
prediction_gpc = predict_model(gpc1, VAL)
prediction_gpc

In [ ]:
mlp1 = create_model('mlp')
finalize_model(mlp1)
save_model(mlp1, './original_model3/mlp1')
prediction_mlp = predict_model(mlp1, VAL)
prediction_mlp

In [ ]:
ridge1 = create_model('ridge')
finalize_model(ridge1)
save_model(ridge1, './original_model3/ridge1_1')
prediction_ridge = predict_model(ridge1, VAL)
prediction_ridge

In [ ]:
rf1 = create_model('rf')
tuned_rf1 = tune_model(rf1)
final_rf1 = finalize_model(tuned_rf1)
save_model(rf1, './original_model3/rf1')
prediction_rf = predict_model(rf1, VAL)
prediction_rf

In [ ]:
qda1 = create_model('qda')
finalize_model(qda1)
save_model(qda1, './original_model3/rf1')
prediction_qda = predict_model(qda1, VAL)
prediction_qda

In [ ]:
# 21%
ada1 = create_model('ada')
finalize_model(ada1)
save_model(ada1, './original_model3/ada1')
prediction_ada = predict_model(ada1, VAL)
prediction_ada

In [ ]:
gbc1 = create_model('gbc')
finalize_model(gbc1)
save_model(gbc1, './original_model3/gbc1')
prediction_gbc = predict_model(gbc1, VAL)
prediction_gbc

In [ ]:
lda1 = create_model('lda')
finalize_model(lda1)
save_model(lda1, './original_model3/lda1')
prediction_lda = predict_model(lda1, VAL)
prediction_lda

In [ ]:
et1 = create_model('et')
finalize_model(et1)
save_model(et1, './original_model3/et1')
prediction_et = predict_model(et1, VAL)
prediction_et

In [ ]:
lightgbm1 = create_model('lightgbm')
finalize_model(lightgbm1)
save_model(lightgbm1, './original_model3/lightgbm1')
prediction_lightgbm = predict_model(lightgbm1, VAL)
prediction_lightgbm